---
title: "Computer Graphics"
author: "Vahram Poghosyan"
date: "2025-05-24"
categories: ["Game Development", "Computer Graphics"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js",
              "OrbitControls": "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js",
              "Cannon": "https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-1.js"></script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-2.js"></script>
---

# OpenGL/WebGL & Shaders

OpenGL is a cross-platform graphics API that is widely used in video game development and computer graphics (it's a competitor to Nvidia's DirectX). WebGL is a [language binding](https://en.wikipedia.org/wiki/Language_binding) of OpenGL in JavaScript. I will use WebGL and OpenGL interchangeably throughout these posts. It provides a set of functions for rendering 2D and 3D graphics, allowing developers to create complex visual effects and realistic environments. GLSL is part of OpenGL. In the Three.js post [Three.js: Introduction to 3D Graphics](../../visualization/three_js_in_jupyter/three_js_in_jupyter.ipynb), we learned how to create a simple 3D scene using Three.js, which is a JavaScript library that abstracts the use of OpenGL API. It uses OpenGL's API calls, internally, to compile, link, and send our GLSL shader code to the GPU.

In this post, we will explore how to use shaders in Three.js to create custom visual effects. Shaders are small programs that run on the GPU and are used to control the rendering of graphics. They allow developers to manipulate the appearance of objects in a scene, such as their color, texture, and lighting. Shaders are separate from the [*rendering pipeline*](#the-three-stages-of-the-rendering-pipeline) (see below), they can be thought of as ad-hoc programs that run on the GPU in a massively parallel way. They can run at *any* point of the rendering pipeline, and do so totally independently of it. Because of this, shaders give us fine-grained control over the rendering process.

Let's create a new scene containing a simple quadrilateral mesh (created using Three.js, which is an abstraction layer over OpenGL). We will use this quad mesh as our canvas to draw on (with shaders). First, we'll stick to 2D. We'll create a vertex shader that takes the vertices of our quadrelateral and maps them to the full screen. The fragment shader will contain most of the magic, for now. It will essentially paint the surface of the quad... 

<details><summary>Click to expand the code used to create a basic scene</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm';

/* Scene / Camera / renderer ---------------------------------------------- */ 
const bodyWidth = document.getElementById("quarto-document-content").clientWidth;
const bodyHeight = 600;
const canvas = document.getElementById("three-d-canvas");

const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

/* Lights ---------------------------------------------- */
scene.add(new three.AmbientLight(0xffffff, 1));
const dir = new three.DirectionalLight(0xffffff, 1);
dir.position.set(10, 20, 10);
scene.add(dir);

/* Plane  ---------------------------------------------- */
const quadMesh = new three.Mesh(
    new three.PlaneGeometry(250, 250), // Plane is added to the XY plane (x+ = right, y+ = up, z+ = out of the screen)
);
scene.add(quadMesh); 

/* Camera ---------------------------------------------- */
camera.position.set(0, 0, 200); // The z+ represetns 100 units towards the viewer (out of the screen)
camera.lookAt(quadMesh.position);

renderer.render(scene, camera);
```

</details>

This is pretty much it for the CPU-side code that creates the scene, camera, and renderer. The next step is to create the shaders.

Final Result:

In [1]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-1'></canvas>"))

Here it is, our empty canvas. Let's add some color to it. 

## Shader Code (GLSL)

As mentioned above, the shader code is written in GLSL (OpenGL Shading Language). The *vertex shader* is responsible for transforming the vertices of the geometry. The fragment shader is responsible for determining the color of each pixel. Note that the fragment shader is also sometimes called *the pixel shader*.

First, we create a vertex shader in the `/shaders` subdirectory of this page. Then, a fragment shader. We import these shaders into our JavaScript code and use them to create a `ShaderMaterial`. Finally, we apply this material to our quad mesh.

```js
import vertexShader from 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.vert?raw';
import fragmentShader from 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.frag?raw';
const shaderMaterial = new three.ShaderMaterial({
    vertexShader: vertexShader,
    fragmentShader: fragmentShader,
    uniforms: {
        uTime: { value: 0.0 },
        uResolution: { value: new three.Vector2(bodyWidth, bodyHeight) }
    }
});

quadMesh.material = shaderMaterial;
```

Note, if we add empty shaders to an object in our canvas we will see pitch black and the following error will be printed to the console:

```
Program Info Log: Vertex shader is not compiled.
```
This is expected, because we haven't written any code in our vertex shader yet. Let's code up our shaders! 

### GLSL Basics

#### Main Function

Every shader has a main function, which is the entry point for the shader code. We can write more functions, which main will call.

```glsl
void otherFunction() {
    // More code here
}

void main() {
    // Your code here
}
```
#### Types

GLSl is strongly typed, meaning we need to declare the types of our variables. The most common types are `float`, `int`, `vec2`, `vec3`, and `vec4`.

- A `float` variable needs to explicitly have a decimal point, e.g. `1.0` or `3.14` or `2.` (otherwise it's interpreted as an `int`).
- All `vec` types are vectors, which are arrays of floats. For example, `vec2` is a 2D vector (array of 2 floats). Vectors can be initialized using the `vec2(x, y)` constructor, where `x` and `y` are the components of the vector. Initializing based on one parameter will create a vector with all components equal to that parameter, e.g. `vec2(1.0)` will create a vector with both components equal to `1.0`.

#### Reading Vectors

We can read the components of a vector using the `x`, `y`, `z`, and `w` properties (`w` is the translation in homogeneous coordinates). If the vector represents a color, we can also use `r`, `g`, `b`, and `a` properties (where `a` is the opacity value). 

Given the vector `vec4 v = vec4(0.1,0.2,0.3,0.4)`, we can access its components as follows:

| 0.1 | 0.2 | 0.3 | 0.4 |
|-----|-----|-----|-----|
| `v.x`   | `v.y`   | `v.z`   | `v.w`   |
| `v.r`   | `v.g`   | `v.b`   | `v.a`   |
| `v.s`   | `v.t`   | `v.p`   | `v.q`   |

We can also access multiple components at once using the `xy`, `xyz`, and `xyzw` properties. For example, `v.xy` will return a `vec2` with the first two components of `v`, i.e. `vec2(0.1, 0.2)`. This is called vector *swizzling*.

Swizzling use cases include:
- Extracting a 2D vector from a 3D vector (such as for simple projection), e.g. `vec3 v = vec3(1.0, 2.0, 3.0); vec2 v2 = v.xy;` will create a `vec2` with the first two components of `v`.
- Swapping color channels, e.g. `vec4 color = vec4(1.0, 0.0, 0.0, 1.0); vec4 swappedColor = color.bgra;` will create a new color with the blue channel in the first position, green in the second, red in the third, and alpha in the fourth.

This is a very powerful way to go hop from dimension into dimension, and to manipulate vectors in a very flexible way. Say we wanted to go from 2D to 3D and we didn't care about the z-coordinate, we could do this:

```glsl
vec2 v1 = vec2(1.0, 2.0);
vec3 v2 = v1.xyx // This will initialize a 3D vector with its z-coordinate equal to the x-coordinate of the 2D vector.
```


#### Attributes, Uniforms, and Varying

Just as we can instantiate variables inside our GLSL shader code, some are also passed into our shaders from the external OpenGL context. These are called *attributes*, and *uniforms*.

We've already seen how to pass uniforms into our shaders using Three.js's `ShaderMaterial`. Here's a quick recap:

```js
const shaderMaterial = new three.ShaderMaterial({
    vertexShader: vertexShader,
    fragmentShader: fragmentShader,
    uniforms: {
        uTime: { value: 0.0 },
        uResolution: { value: new three.Vector2(bodyWidth, bodyHeight) }
    }
});
```
The above code passes two uniforms into our shaders: `uTime` and `uResolution`. These can be used in both the vertex and fragment shaders to control the rendering process. For example, `uTime` can be used to create animations, while `uResolution` can be used to adjust the rendering based on the size of the HTML canvas.

The difference between attributes and uniforms is that attributes are per-vertex data, while uniforms are global data that is shared across all vertices and fragments. See table below for a summary of the differences:

|              | Attribute      | Uniform      |
|--------------|---------------|-------------|
| Available in Vertex Shader?      | Read Only      | Read only   |
| Available in Fragment Shader?    | N.A           | Read only   |
| Set From     | CPU           | CPU         |
| Contain Information      | Per Vertex    | Constant    |

There are also *varying* variables, which are used to pass data from the vertex shader to the fragment shader (because they're the only variables that are writable to). Varying variables are written to inside the vertex shader and passed as read-only to the fragment shader. They are commonly used for interpolating values across the surface of a mesh. For example, if we want to pass the color of a vertex to the fragment shader, we can declare a varying variable in the vertex shader and assign it a value. Then, in the fragment shader, we can read that varying variable to get the interpolated color for each pixel.

Here's the table summarizing the differences between attributes, uniforms, and varying variables:

|              | Attribute      | Uniform      | Varying      |
|--------------|---------------|-------------|-------------|
| Available in Vertex Shader?      | Read Only      | Read only   | Read/Write  |
| Available in Fragment Shader?    | N.A           | Read only   | Read Only   |
| Set From     | CPU           | CPU         | Vertex Shader |
| Contain Information      | Per Vertex    | Constant    | Per Fragment |

Attributes contain information per vertex (i.e. data about each vertex, such as its color) because they're passed to the vertex shader as variables (which works on vector geometry -- i.e. vertices). Varying Variables, on the other hand, are used to pass data from the vertex shader down to the fragment shader, which works with fragments, hence they contain information *per fragment*.

#### Normalized Vectors

In GLSL, vectors are often normalized to have a length of 1. This is useful for many operations, such as lighting calculations and texture mapping. Normalizing a vector is done using the `normalize()` function. For example, if we have a vector `vec3 v = vec3(1.0, 2.0, 3.0);`, we can normalize it by calling `vec3 normalizedV = normalize(v);`. This will create a new vector with the same direction as `v`, but with a length of 1.

Individual entries are normalized too. For example white is represented as `vec3(1.0, 1.0, 1.0)`, rather than `vec3(255, 255, 255)`. Note, this vector is not normalized. But *normalization* is also often used in this sense.

### Writing Our First Basic Shaders

Let's start with a fragment shader that maps all pixels to a single color. The function `main()` below runs per each pixel and sets their value to purple. We use this to simply color the entire quad purple.

```glsl
void main() {
    // Make every pixel purple
    gl_FragColor = vec4(0.5, 0.0, 0.5, 1.0); // Set the color to purple (RGBA)
}
```

Now, let's write our first vertex shader. It will simply stretch out the rectangle to cover the entire screen. Then, we will pass the final vertex positions to the fragment shader, which will use them to color the pixels.

As mentioned before, the vertex shader receives attributes (per vertex data). These include:

```glsl
attribute vec3 aPosition;
attribute vec2 aTexCoord;
```

- `aPosition` is the position (relative to world space) of the vertex in 3D space (x, y, z). Our vertex shader will run once per vertex, and it will receive the position of that vertex as an attribute.
- `aTexCoord` is the texture coordinate of the vertex (u, v). This is used to map textures onto the geometry in the fragment shading stage. This is the coordinate we would use in the fragment shader to draw on the pixel corresponding to this vertex.

These attributes are automatically set by Three.js (via the OpenGL API, internally).

In our vertex shader we define a Varying Variable `pos`, which will be used to pass the texture coordinates (`aTexCoord`) to the fragment shader. Essentially, after the vertex shader is done scrambling the vertices, the new texture coordinates of these scrambled vertices are passed down to the fragment shader.

Let's stretch the quad to cover the entire screen.

<details><summary>Click to expand the vertex shader code</summary>

```glsl
attribute vec3 aPosition;
attribute vec2 aTexCoord;

varying vec2 pos;

void main() {
    pos = aTexCoord;

    vec4 position = vec4(aPosition, 1.0);
    position.xy = position.xy * 2.0 - 1.0; // Map the positions from the [0,1] range to the [-1,1] range

    gl_Position = position;
}
```

</details>


This gives:

In [2]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-2'></canvas>"))

# The Three Main Stages of the Rendering Pipeline

After some CPU side pre-processing, the rendering pipeline consists of three main stages:

## Vertex Shading

The first stage of the rendering pipeline is the vertex shading stage. In this stage, the vertex shader is executed for each vertex of the geometry. The vertex shader is responsible for transforming the vertices of the geometry from object space to clip space. It can also be used to perform other operations, such as calculating normals or texture coordinates.

### World Matrix, Local Matrix, and Model Matrix

This is where three crucial matrices come into play: the *model matrix*, the *view matrix*, and the *projection matrix*. The model matrix transforms the vertices from object space to world space, the view matrix transforms the vertices from world space to camera space, and the projection matrix transforms the vertices from camera space to clip space. The final position of each vertex is calculated by multiplying these three matrices together. As always with matrices, the order of multiplication matters. The final position of each vertex is given by:

The position of a given vertex $\mathbf{v}$ in clip space is:

$$
\mathbf{v}_{\text{clip}} = \mathbf{P} \cdot \mathbf{V} \cdot \mathbf{M} \cdot \mathbf{v}
$$

Once the position of each vertex is calculated, a *primitives assembly* stage is performed, where the vertices are grouped into primitives (e.g., triangles, lines, etc.). The primitives are then rasterized to generate fragments, which are the pixels that will be drawn on the screen.

Geometry shading is a more advanced stage that can be used to generate new primitives based on the existing primitives. It is not always used, but it can be useful for certain effects, such as generating shadows or reflections.

A *tessellation stage* can also be used to subdivide the primitives into smaller ones, allowing for more detailed rendering. This is often used in high-end graphics applications, such as video games and simulations.

## Rasterization

The magical step where vector-based primitives are converted into fragments (pixels). This is where the geometry is transformed into a 2D representation that can be displayed on the screen. The rasterization stage takes the primitives generated in the previous stage and converts them into fragments, which are the pixels that will be drawn on the screen. Each fragment is assigned a color based on the lighting and shading calculations performed in the next stage.

Rasterization and *Ray-Tracing* (described briefly in the Three.js post [Three.js: Introduction to 3D Graphics](../../visualization/three_js_in_jupyter/three_js_in_jupyter.ipynb)) are two different approaches. Rasterization is a brute-force and more common approach, while ray-tracing is more accurate and produces more realistic images. Ray-tracing is often used in high-end graphics applications, such as movies and animations, where the quality of the image is more important than the speed of rendering. Rasterization, on the other hand, is used in real-time applications, such as video games, where speed is more important than quality. However, this is changing with the advent of modern GPUs that can perform ray-tracing in real-time.


What's the difference? Rasterization and [ray-tracing](https://en.wikipedia.org/wiki/Ray_tracing_(graphics)) are two different algorithms that both, essentially, take *primitives* (which are vector based) and output *fragments* (which are the last abstraction layer before a pixel color value). The value of a given pixel is determined from the fragment that covers that pixel (and on whether other fragments are in front of *it* and the camera, at least in the case of rasterization). To understand the surface-level difference between rasterization and ray tracing, think in terms of two nested loops. Each object covers a certain area of pixels on the screen, so the rasterization algorithm takes each object first and determines which pixels it covers. Then, for each pixel it determines if the object is the closest one to the given pixel or not. Ray tracing, on the other hand, flips the loops. It asks, for each pixel, which object is the closest one to it. It does this by casting rays from the camera into the scene and checking for intersections with objects. 

Here's a great [video](https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s) from Nvidia that explains the difference between rasterization and ray-tracing in more detail. I will embed it here.

{{< video https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s >}}

Raytracing can be used in conjunction with rasterization to achieve a balance between quality and performance. For example, ray-tracing can be used to calculate reflections and refractions, while rasterization can be used for the rest of the scene. This is often referred to as *hybrid rendering*. For example, *screen-space reflections (SSR)* is a technique that uses ray-tracing to calculate reflections in a scene, while rasterization is used for the rest of the scene. This allows for more realistic reflections without the performance overhead of full ray-tracing. *Ambient occlusion* is another technique that uses ray-tracing to calculate the amount of light that reaches a surface, while rasterization is used for the rest of the scene. While these can be thought of as *post-processing effect* I like to reserve the term *post-processing* for effects that are applied after the entire scene has been drawn and colored (i.e. after *fragment shading*). These effects include: *bloom*, *motion blur*, *depth of field*, *aliasing*, etc. However, after rasterization we already have a colored scene, more or less, so I can see why the term post-processing can still apply. Given that *fragment shading* and *post-processing* are at the same level in the pipeline, it's a matter of preference whether to call these effects post-processing or not. *Post-processing* is *fragment shading* (in essence).

## Fragment Shading

The final stage of the rendering pipeline is the fragment shading stage. In this stage, the fragment shader is executed for each fragment generated in the rasterization stage. The fragment shader is responsible for determining the color of each pixel based on the lighting and shading calculations performed in the previous stages. It can also be used to apply *textures*, perform *post-processing* effects, and more.

A key object in this stage is the *frame buffer*, which is a memory buffer that stores the color and depth information of each pixel. A *Frame Buffer Object (FBO)* is an object that's used to store the shader calculations on a given scene to a texture rather than to the screen itself. This allows us to overlay post-processing effects. Each effect is drawn to its own *texture* within a separate FBO. What's shown on the screen is, then, all of these *textures* (along with other shader calculations performed during the fragment shading stage) overlaid on top of the rasterized scene.

# Euler Angles and Rotation Matrices

## Intertial Frame and Body Frame

$R_{yaw}$, $R_{pitch}$, and $R_{roll}$.

# Quaternion Rotation